This file takes in a user-defined class with a certain name. Instantiating the network is simply for testing purposes

In [1]:
import importlib.util
import os
import torch
from torch import nn
from torch.utils.data import DataLoader

In [2]:
class NetworkAnalyzer:

    def __init__(self, model_architecture: nn.Module, amount_to_produce: int, success_criteria: callable = None, max_attempts: float = None):
        """
        This is the constructor to out network analyzer

        Parameters: 
            model (nn.Module): PyTorch class specifying the Neural Network Architecture.
            amount_to_produce (int): How many good Neural Networks to produce.
            success_criteria (callable): Lambda expression that sets a user-defined criteria for the loss.\
                The default is 10% loss.
            max_attempts: Maximum amount of times a network should be instantiated in case an instance\
                does not meet the success criteria
        """
        self.model_architecture = model_architecture
        self.amount_to_produce = amount_to_produce
        self.success_criteria = success_criteria if success_criteria is not None else self.default_success_criteria()
        self.max_attempts = max_attempts if max_attempts is not None else self.default_max_attempts()
    
    def default_success_criteria(self):
        """
        Makes the success criteria be a 10% loss if not specified by the user

        Returns:    
            Predefined 10% test-loss as success criteria
        """
        return 10
    
    def default_max_attempts(self):
        """
        Makes it so that the maximum attempts is an extra 30% over the number of networks to produce,\
        if not specified by the user

        Returns:
            The number of max attempts
        """
        return int(self.amount_to_produce * 1.3)
    
    def show_loss(epoch, loss_value):
        """
        This function prints the loss, and is used by the training function.
        This is its own function so it can be used by the user if they 
        wish to define their own training function.
        """
        print(f"Epoch [{epoch+1}], Loss: {loss_value:.4f}")
    
    def train_network(self, X_train, y_train, X_test, y_test, network: nn.Module, num_epochs, optimizer: torch.optim.SGD):
        """
        Function that trains the network

        Parameters:
            X_train: Data to train the model
            Y_train: Labels of the training data
            X_test: Data to test the model
            Y_test: Labels of the testing data
        """
    
        criterion = nn.MSELoss() # Don't like
        optimizer = torch.optim.SGD(network.parameters(), lr = 0.01)
        for epoch in range(num_epochs):
            network.train()
            # Forward Pass
            outputs = network(X_train)
            loss = criterion(outputs, y_train)

            # Backprop and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        self.show_loss(epoch, loss.item())

    
    
    def generate_networks(self):

        self.models = []

        for i in range(self.amount_to_produce):
            self.models.append()

        print(f"{len(instances)} instances created.")
    


    

In [ ]:
def train_network2(self, model: nn.Module, train_loader: DataLoader, test_loader: DataLoader, num_epochs: int, optimizer: torch.optim.SGD, loss_fn: torch.nn):
        """
        Function that trains the network

        Parameters:
            
        """
    
        for epoch in range(num_epochs):
            for batch in train_loader:
                optimizer.zero_grad()
                input, target = batch
                output = model(input)
                loss = loss_fn(output, target)
                loss.backward()
                optimizer.step()

In [ ]:
def train_model(self, model: nn.Module, train_loader: DataLoader, loss_fn: torch.nn, optimizer: torch.optim.SGD, num_epochs: int):
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0

        for i, (inputs, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accumulate loss
            running_loss += loss.item()

        # Print loss after every epoch
        epoch_loss = running_loss / len(train_loader)
        self.print_loss(epoch, epoch_loss)